In [1]:
# https://pyimagesearch.com/2018/06/04/keras-multiple-outputs-and-multiple-losses/

import numpy as np
import random
import os

# ! pip install split-folders
from imutils import paths
import cv2
import pandas as pd

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.preprocessing import image


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

import matplotlib.pyplot as plt

rs = 42
def reset_random_seeds(rs):
   os.environ['PYTHONHASHSEED']=str(rs)
   tf.random.set_seed(rs)
   np.random.seed(rs)
   random.seed(rs)
reset_random_seeds(rs)

map_dir = 'map 5'

# Part 1 - Data Preprocessing

## Split into train , test data

In [2]:
header_list = ['collaborative_value','computer_value','predictable_value','selfishly_value','wisely_value']
rate_vectors = pd.read_csv('./data/humanModel_v1_dataset/target.csv', names=header_list)

In [3]:
rate_vectors

,collaborative_value,computer_value,predictable_value,selfishly_value,wisely_value
0,5,6,5,5,4
1,5,6,5,5,4
2,5,6,5,5,4
3,5,6,5,5,4
4,5,6,5,5,4
...,...,...,...,...,...
5589,7,4,7,1,4
5590,7,4,7,1,4
5591,7,4,7,1,4
5592,7,4,7,1,4


In [4]:
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
# imagePaths = sorted(list(paths.list_images('./data/humanModel_v1_dataset/')))
# random.shuffle(imagePaths)
imagePaths = list(paths.list_images('./data/humanModel_v1_dataset/'))

# initialize the data
data = []
actionLabels = []
rate_vector_value = []

[INFO] loading images...


### Preprocessing the Training set

In [5]:
batch_size = 64
IMG_SIZE = 10

# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	# image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = img_to_array(image)
	data.append(image)
	# extract the action and rate from the path and
	# update the respective lists
	split_path = imagePath.split(os.path.sep)
	action = split_path[-2]
	actionLabels.append(action)
	index = int(split_path[-1].split("_")[-1].split(".")[0])
	value_rate = rate_vectors.iloc[index].tolist()
	rate_vector_value.append(value_rate)

In [6]:
# scale the raw pixel intensities to the range [0, 1] and convert to
# a NumPy array
data = np.array(data, dtype="float") / 255.0
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
	len(data), data.nbytes / (1024 * 1000.0)))
# convert the label lists to NumPy arrays prior to binarization
actionLabels = np.array(actionLabels)
rate_vector_value = np.array(rate_vector_value) / 7
# binarize both sets of labels
print("[INFO] binarizing labels...")
actionLB = LabelBinarizer()
actionLabels = actionLB.fit_transform(actionLabels)

# partition the data into training and testing splits using 80% of
split = train_test_split(data, actionLabels, rate_vector_value, test_size=0.2, random_state=rs, shuffle=True)
(trainX, testX, trainActionY, testActionY, trainRateY, testRateY) = split

[INFO] data matrix: 5594 images (13.11MB)
[INFO] binarizing labels...


In [7]:
from tensorflow.keras.layers import LSTM, TimeDistributed

RATE_VECTOR_LEN = 5

# Define model layers.
input_layer = Input(shape=(IMG_SIZE,IMG_SIZE,3))
X = Conv2D(8, 4, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(input_layer)
X = Conv2D(16, 4, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
X = Conv2D(16, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
X = MaxPooling2D()(X)
X = Conv2D(8, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
X = Dropout(0.5)(X)
X = Flatten()(X)
X = Dense(40, activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
X = LSTM(32)(X)
rate_output = Dense(RATE_VECTOR_LEN, activation='linear', name='rate_output')(X)

# Define the model with the input layer 
# and a list of output layers
model = Model(inputs=input_layer,outputs=rate_output)


ValueError: Input 0 of layer lstm is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 40)

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 20
batch_size = 5

# Fit the model to the training data.
model.fit(
    trainX,
    trainRateY,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(testX, testRateY),
    callbacks=[early_stopping, reduce_lr],
)

## View some data

In [ ]:
ACTION_NUM = 5
RATE_VECTOR_LEN = 5

# Define model layers.
input_layer = Input(shape=(IMG_SIZE,IMG_SIZE,3))
X = Conv2D(8, 4, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(input_layer)
X = Conv2D(16, 4, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
X = Conv2D(16, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
X = MaxPooling2D()(X)
X = Conv2D(8, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
X = Dropout(0.5)(X)
X = Flatten()(X)
X = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)

# action output will be fed from the first dense
X = Dense(20, activation='relu', kernel_regularizer=regularizers.l2(0.001))(X)
rate_output = Dense(RATE_VECTOR_LEN, activation='linear', name='rate_output')(X)

action_output = Dense(ACTION_NUM, activation='softmax', name='action_output')(rate_output)

# Define the model with the input layer 
# and a list of output layers
model = Model(inputs=input_layer,outputs=[action_output, rate_output])

In [ ]:
model.summary()

In [ ]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 350
INIT_LR = 1e-3

# define two dictionaries: one that specifies the loss method for
# each output of the network along with a second dictionary that
# specifies the weight per loss
	
losses = {
	"action_output": "categorical_crossentropy",
	"rate_output": "mse"
}
lossWeights = {"action_output": 0.5, "rate_output": 0.5}

# initialize the optimizer and compile the model
print("[INFO] compiling model...")

opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights,
	metrics=["accuracy"])

# model.compile(optimizer='adam',
#               loss=losses,
# 			  loss_weights=lossWeights,
#               metrics=['accuracy'])


In [ ]:
# train the network to perform multi-output
history = model.fit(x=trainX, y={"action_output": trainActionY, "rate_output": trainRateY},
	validation_data=(testX, {"action_output": testActionY, "rate_output": testRateY}),
	epochs=EPOCHS)

In [ ]:
acc = history.history['action_output_accuracy']
val_acc = history.history['val_action_output_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
_ = model.evaluate(x=testX, y={'action_output': testActionY, 'rate_output': testRateY})
# "coll_output": testCollY, "comp_output": testCompY, "pred_output": testPredY, "self_output": testSelfY, "wise_output": testWiseY})

In [ ]:
# Save the entire model as a SavedModel.
model.save('./data/'+map_dir+'/humanModel_v2.h5')

In [ ]:
# new_model = tf.keras.models.load_model('./data/humanModel/mode_v0')

# # Check its architecture
# new_model.summary()

### Predict on new data

In [ ]:
path = "./data/humanModel_v1_dataset/38/-Ms8av2Z5M2XEIGMqjsx_70.png"

img = keras.preprocessing.image.load_img(
    path, target_size=(IMG_SIZE, IMG_SIZE)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
img_array = img_array / 255.0
predictions = model.predict(img_array)

print(f"The vector rate: {predictions[1]}")
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(np.argmax(predictions[0]), 100 * np.max(predictions[0]))
)